#### Core AutoGen
- Model
- Messages
- Agents
- Teams

In [15]:
## loading the env variable 
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [16]:
## Model definition
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [17]:
## Message 
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(id='1e212a1d-bf44-45b7-bbf2-8382eda93de5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 7, 13, 33, 2, 23950, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

In [18]:
## Agents Define 
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

In [19]:
## try out
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

"Sure! Just remember to pack your umbrella—London's idea of a sunny day is when it only drizzles! 🐑☔✈️"

In [21]:
## local storage example
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("db/tickets.db"):
    os.remove("db/tickets.db")

# Create the database and the table
conn = sqlite3.connect("db/tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [22]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("db/tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [ ]:
# Method to get price for a city -> tool
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("db/tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [27]:
get_city_price("Rome")

499.0

In [28]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [29]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_WYLcESl3M4DGBOd6KMVnwCzM', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_WYLcESl3M4DGBOd6KMVnwCzM', is_error=False)]


'Sure! A roundtrip ticket to London is just $299. If only they had a "Teleportation" service, we\'d be charging just 29 cents!'